In [27]:
!pip -q install pandas numpy

In [28]:
import numpy as np
import pandas as pd

np.random.seed(42)

# -----------------------------
# CONFIG
# -----------------------------
START_DATE = "2023-01-01"
END_DATE   = "2024-12-31"   # 2 years daily
N_STORES   = 6
N_PRODUCTS = 40

dates = pd.date_range(START_DATE, END_DATE, freq="D")
n_days = len(dates)

# -----------------------------
# STORES TABLE
# -----------------------------
store_regions = ["Northeast", "South", "Midwest", "West"]
stores = pd.DataFrame({
    "store_id": [f"S{i:03d}" for i in range(1, N_STORES+1)],
    "region": np.random.choice(store_regions, size=N_STORES),
    "store_size": np.random.choice(["Small", "Medium", "Large"], size=N_STORES, p=[0.3, 0.5, 0.2]),
})

region_multiplier = {"Northeast": 1.00, "South": 1.08, "Midwest": 0.95, "West": 1.03}
size_multiplier = {"Small": 0.85, "Medium": 1.00, "Large": 1.20}

stores["region_mult"] = stores["region"].map(region_multiplier)
stores["size_mult"] = stores["store_size"].map(size_multiplier)
stores["store_mult"] = stores["region_mult"] * stores["size_mult"]

# -----------------------------
# PRODUCTS TABLE
# -----------------------------
categories = ["Beverages", "Snacks", "Dairy", "Frozen", "Household"]
products = pd.DataFrame({
    "product_id": [f"P{i:04d}" for i in range(1, N_PRODUCTS+1)],
    "category": np.random.choice(categories, size=N_PRODUCTS),
})

# baseline demand per product (units/day) and price
products["base_demand"] = np.random.uniform(8, 60, size=N_PRODUCTS)  # average daily demand
products["base_price"]  = np.random.uniform(2.5, 25, size=N_PRODUCTS)

# price elasticity per product (negative): larger magnitude = more sensitive
products["price_elasticity"] = -np.random.uniform(0.8, 2.2, size=N_PRODUCTS)

# lead time distribution per product (days)
products["lead_time_mean"] = np.random.choice([3, 5, 7, 10], size=N_PRODUCTS, p=[0.25, 0.35, 0.25, 0.15])
products["lead_time_sd"]   = np.random.choice([1, 2, 3], size=N_PRODUCTS, p=[0.5, 0.35, 0.15])

# -----------------------------
# HOLIDAYS / EVENTS (simple realistic set)
# -----------------------------
def is_holiday(d):
    # US-like major retail demand bumps
    month, day = d.month, d.day
    if (month == 11 and 20 <= day <= 30):  # Thanksgiving/Black Friday window
        return "ThanksgivingWindow"
    if (month == 12 and 10 <= day <= 31):  # Holiday season
        return "HolidaySeason"
    if (month == 7 and 1 <= day <= 7):     # July 4 week
        return "July4Week"
    if (month == 2 and 7 <= day <= 14):    # Super Bowl / Valentine's-ish
        return "FebEvent"
    return "None"

calendar = pd.DataFrame({"date": dates})
calendar["dow"] = calendar["date"].dt.dayofweek  # 0=Mon
calendar["is_weekend"] = (calendar["dow"] >= 5).astype(int)
calendar["month"] = calendar["date"].dt.month
calendar["weekofyear"] = calendar["date"].dt.isocalendar().week.astype(int)
calendar["holiday_event"] = calendar["date"].apply(is_holiday)

holiday_uplift = {
    "None": 1.00,
    "FebEvent": 1.06,
    "July4Week": 1.10,
    "ThanksgivingWindow": 1.15,
    "HolidaySeason": 1.20
}

# -----------------------------
# MAIN FACT TABLE GENERATION
# daily_sales: store x product x day
# -----------------------------
rows = []
for _, s in stores.iterrows():
    for _, p in products.iterrows():

        # product-store baseline
        base = p["base_demand"] * s["store_mult"]

        # store-product noise scale
        noise_sd = np.random.uniform(1.5, 6.0)

        # promo cadence: each product-store has occasional promos
        promo_prob = np.random.uniform(0.03, 0.08)  # 3% to 8% of days
        promo_flag = (np.random.rand(n_days) < promo_prob).astype(int)

        # discount only on promo days
        discount_pct = np.where(promo_flag == 1, np.random.uniform(0.05, 0.35, size=n_days), 0.0)

        # price varies around base price + some day-level randomness
        # promo reduces price
        raw_price = p["base_price"] * (1 + np.random.normal(0, 0.04, size=n_days))
        price = raw_price * (1 - discount_pct)
        price = np.clip(price, 0.5, None)

        # seasonality: weekly + yearly-ish
        # weekly: weekends higher for many consumer categories
        weekend_mult = 1.08 if p["category"] in ["Snacks", "Beverages", "Frozen"] else 1.03
        weekly = 1 + (calendar["is_weekend"].values * (weekend_mult - 1))

        # monthly seasonality: mild
        month = calendar["month"].values
        monthly = 1 + 0.08*np.sin(2*np.pi*(month-1)/12)

        # holiday uplift
        holiday_mult = calendar["holiday_event"].map(holiday_uplift).values

        # price elasticity effect:
        # demand multiplier = (price / base_price) ^ elasticity
        # elasticity is negative -> higher price reduces demand
        price_mult = (price / p["base_price"]) ** (p["price_elasticity"])

        # promo lift (beyond price): e.g., merchandising effect
        promo_mult = 1 + promo_flag * np.random.uniform(0.08, 0.25)

        # supply disruption random days (e.g., late truck), reduce available inventory
        disruption = (np.random.rand(n_days) < np.random.uniform(0.01, 0.03)).astype(int)

        # True demand (what customers would buy if always in stock)
        demand_true = base * weekly * monthly * holiday_mult * price_mult * promo_mult
        demand_true = demand_true + np.random.normal(0, noise_sd, size=n_days)
        demand_true = np.maximum(demand_true, 0).round().astype(int)

        # Inventory simulation: on-hand is replenished weekly-ish, can cause stockouts
        # Simple policy: every Monday receive some inventory based on last week's average demand
        on_hand = 0
        sales_units = np.zeros(n_days, dtype=int)
        stockout_flag = np.zeros(n_days, dtype=int)
        on_hand_end = np.zeros(n_days, dtype=int)

        for i, d in enumerate(dates):
            # Monday replenishment (dayofweek=0)
            if d.dayofweek == 0:
                # order quantity based on recent demand
                recent = demand_true[max(0, i-14):i]  # 2-week lookback
                avg_recent = recent.mean() if len(recent) > 0 else base
                order_qty = int(np.ceil(avg_recent * 7 * np.random.uniform(0.8, 1.2)))

                # disruption reduces received quantity sometimes
                if disruption[i] == 1:
                    order_qty = int(order_qty * np.random.uniform(0.4, 0.8))

                on_hand += order_qty

            # sales capped by inventory
            sold = min(on_hand, demand_true[i])
            sales_units[i] = sold
            if sold < demand_true[i]:
                stockout_flag[i] = 1

            on_hand -= sold
            on_hand_end[i] = on_hand

        df = pd.DataFrame({
            "date": dates,
            "store_id": s["store_id"],
            "product_id": p["product_id"],
            "category": p["category"],
            "price": price.round(2),
            "promo_flag": promo_flag,
            "discount_pct": discount_pct.round(2),
            "holiday_event": calendar["holiday_event"].values,
            "is_weekend": calendar["is_weekend"].values,
            "demand_true": demand_true,
            "sales_units": sales_units,
            "stockout_flag": stockout_flag,
            "inventory_on_hand_end": on_hand_end
        })
        rows.append(df)

daily_sales = pd.concat(rows, ignore_index=True)

print("daily_sales shape:", daily_sales.shape)
daily_sales.head()

daily_sales shape: (175440, 13)


,date,store_id,product_id,category,price,promo_flag,discount_pct,holiday_event,is_weekend,demand_true,sales_units,stockout_flag,inventory_on_hand_end
0,2023-01-01,S001,P0001,Dairy,4.52,0,0.00,None,1,35,0,1,0
1,2023-01-02,S001,P0001,Dairy,5.17,0,0.00,None,0,27,27,0,190
2,2023-01-03,S001,P0001,Dairy,5.16,0,0.00,None,0,32,32,0,158
3,2023-01-04,S001,P0001,Dairy,4.31,1,0.13,None,0,48,48,0,110
4,2023-01-05,S001,P0001,Dairy,5.22,0,0.00,None,0,31,31,0,79


In [29]:
daily_sales.to_csv("daily_sales.csv", index=False)
products.to_csv("products.csv", index=False)
stores.drop(columns=["region_mult","size_mult","store_mult"]).to_csv("stores.csv", index=False)

print("Saved: daily_sales.csv, products.csv, stores.csv")

Saved: daily_sales.csv, products.csv, stores.csv


In [30]:
import plotly.express as px

sample = daily_sales.query("store_id=='S001' and product_id=='P0001'").copy()

fig = px.line(sample, x="date", y=["demand_true", "sales_units"], title="Demand vs Sales (Stockouts will show gaps)")
fig.show()

fig2 = px.line(sample, x="date", y="inventory_on_hand_end", title="Inventory on hand (end of day)")
fig2.show()

In [31]:
import pandas as pd

# Load dataset if needed
# daily_sales = pd.read_csv("daily_sales.csv", parse_dates=["date"])

sku = daily_sales.query("store_id=='S001' and product_id=='P0001'").copy()
sku = sku.sort_values("date")

sku.head()

,date,store_id,product_id,category,price,promo_flag,discount_pct,holiday_event,is_weekend,demand_true,sales_units,stockout_flag,inventory_on_hand_end
0,2023-01-01,S001,P0001,Dairy,4.52,0,0.00,None,1,35,0,1,0
1,2023-01-02,S001,P0001,Dairy,5.17,0,0.00,None,0,27,27,0,190
2,2023-01-03,S001,P0001,Dairy,5.16,0,0.00,None,0,32,32,0,158
3,2023-01-04,S001,P0001,Dairy,4.31,1,0.13,None,0,48,48,0,110
4,2023-01-05,S001,P0001,Dairy,5.22,0,0.00,None,0,31,31,0,79


In [32]:
import numpy as np

# Time features
sku["day_of_week"] = sku["date"].dt.dayofweek
sku["month"] = sku["date"].dt.month
sku["weekofyear"] = sku["date"].dt.isocalendar().week.astype(int)

# Lag features
sku["lag_1"] = sku["demand_true"].shift(1)
sku["lag_7"] = sku["demand_true"].shift(7)
sku["lag_14"] = sku["demand_true"].shift(14)

# Rolling mean
sku["rolling_7"] = sku["demand_true"].rolling(7).mean()
sku["rolling_14"] = sku["demand_true"].rolling(14).mean()

# Drop missing rows from lags
sku = sku.dropna()

In [33]:
train = sku[sku["date"] < "2024-07-01"]
test  = sku[sku["date"] >= "2024-07-01"]

In [34]:
!pip -q install xgboost

from xgboost import XGBRegressor

features = [
    "price", "promo_flag", "discount_pct",
    "day_of_week", "month", "weekofyear",
    "lag_1", "lag_7", "lag_14",
    "rolling_7", "rolling_14"
]

X_train = train[features]
y_train = train["demand_true"]

X_test = test[features]
y_test = test["demand_true"]

model = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    random_state=42
)

model.fit(X_train, y_train)

test["prediction"] = model.predict(X_test)

/tmp/ipython-input-431/490551298.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=test["date"], y=y_test,
                         mode='lines', name='Actual'))

fig.add_trace(go.Scatter(x=test["date"], y=test["prediction"],
                         mode='lines', name='Forecast'))

fig.update_layout(title="Forecast vs Actual Demand")
fig.show()

In [36]:
import numpy as np

residuals = y_train - model.predict(X_train)
residual_std = np.std(residuals)

test["upper_95"] = test["prediction"] + 1.96 * residual_std
test["lower_95"] = test["prediction"] - 1.96 * residual_std

/tmp/ipython-input-431/3077713086.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-431/3077713086.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=test["date"], y=test["prediction"],
                         mode='lines', name='Forecast'))

fig.add_trace(go.Scatter(
    x=test["date"],
    y=test["upper_95"],
    mode='lines',
    line=dict(width=0),
    showlegend=False))

fig.add_trace(go.Scatter(
    x=test["date"],
    y=test["lower_95"],
    fill='tonexty',
    mode='lines',
    line=dict(width=0),
    name='95% Interval'))

fig.show()

In [38]:
service_level = 0.95
z = 1.65  # approx for 95%

lead_time = 7  # days

sigma_d = residual_std
mu_d = test["prediction"].mean()

safety_stock = z * sigma_d * np.sqrt(lead_time)
reorder_point = mu_d * lead_time + safety_stock

print("Safety Stock:", round(safety_stock))
print("Reorder Point:", round(reorder_point))

Safety Stock: 4
Reorder Point: 223


In [57]:
# Start fresh
df = daily_sales.copy()
df = df.sort_values(["store_id", "product_id", "date"])

# Lag features
df["lag_1"] = df.groupby(["store_id", "product_id"])["demand_true"].shift(1)
df["lag_7"] = df.groupby(["store_id", "product_id"])["demand_true"].shift(7)
df["rolling_7"] = df.groupby(["store_id", "product_id"])["demand_true"].transform(lambda x: x.rolling(7).mean())

# Time features
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month

# Drop NA from lag
df = df.dropna()

# Automatically detect categorical columns (SAFE METHOD)
cat_cols = df.select_dtypes(include=["object"]).columns

print("Categorical columns being encoded:", cat_cols)

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

print("Encoding successful.")
print("Total columns now:", len(df.columns))

Categorical columns being encoded: Index(['store_id', 'product_id', 'category', 'holiday_event'], dtype='object')
Encoding successful.
Total columns now: 66


In [58]:
train = df[df["date"] < "2024-07-01"].copy()
test  = df[df["date"] >= "2024-07-01"].copy()

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (129600, 66)
Test shape: (44160, 66)


In [59]:
features = [col for col in df.columns if col not in [
    "date",
    "demand_true",
    "sales_units",
    "stockout_flag",
    "inventory_on_hand_end"
]]

print("Number of features:", len(features))

Number of features: 61


In [60]:
X_train = train[features]
y_train = train["demand_true"]

X_test = test[features]
y_test = test["demand_true"]

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train: (129600, 61)
X_test: (44160, 61)


In [61]:
features = [col for col in df.columns if col not in [
    "date",
    "demand_true",
    "sales_units",
    "stockout_flag",
    "inventory_on_hand_end"
]]

print("Number of features:", len(features))

Number of features: 61


In [62]:
X_train = train[features]
y_train = train["demand_true"]

X_test = test[features]
y_test = test["demand_true"]

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train: (129600, 61)
X_test: (44160, 61)


In [63]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

print("Model training complete.")

Model training complete.


In [64]:
train["prediction"] = model.predict(X_train)
test["prediction"]  = model.predict(X_test)

print("Predictions added.")
print("Train rows:", len(train), "Pred rows:", len(train["prediction"]))
print("Test rows:", len(test), "Pred rows:", len(test["prediction"]))

Predictions added.
Train rows: 129600 Pred rows: 129600
Test rows: 44160 Pred rows: 44160


In [65]:
import numpy as np

residuals = y_train - train["prediction"]
residual_std = np.std(residuals)

print("Global Residual Std:", round(residual_std, 3))

Global Residual Std: 4.14


In [66]:
z = 1.65  # approx for 95%

test["upper_95"] = test["prediction"] + z * residual_std
test["lower_95"] = test["prediction"] - z * residual_std

print("Prediction intervals added.")

Prediction intervals added.


In [67]:
import numpy as np
import pandas as pd

# ---- Inputs (interactive later in dashboard)
lead_time_days = 7
service_level = 0.95

# Approx z values (good enough for portfolio)
z_map = {0.90: 1.28, 0.95: 1.65, 0.97: 1.88, 0.99: 2.33}
Z = z_map.get(service_level, 1.65)

# ---- Create a SKU key BEFORE encoding next time (recommended)
# For now, we reconstruct a key from one-hot columns (works, but clunky)
store_cols = [c for c in df.columns if c.startswith("store_id_")]
prod_cols  = [c for c in df.columns if c.startswith("product_id_")]

def decode_onehot(row, cols, prefix):
    # returns the active column name suffix if found, else baseline category (dropped first)
    active = [c for c in cols if row[c] == 1]
    return active[0].replace(prefix, "") if active else f"{prefix}BASE"

tmp = test.copy()

tmp["store_key"] = tmp.apply(lambda r: decode_onehot(r, store_cols, "store_id_"), axis=1)
tmp["prod_key"]  = tmp.apply(lambda r: decode_onehot(r, prod_cols, "product_id_"), axis=1)

policy = (
    tmp.groupby(["store_key", "prod_key"], as_index=False)
       .agg(mu_d=("prediction", "mean"),
            sigma_d=("prediction", "std"))
)

policy["sigma_d"] = policy["sigma_d"].fillna(residual_std)  # fallback if std is NaN
policy["safety_stock"] = Z * policy["sigma_d"] * np.sqrt(lead_time_days)
policy["reorder_point"] = policy["mu_d"] * lead_time_days + policy["safety_stock"]

policy.head()

,store_key,prod_key,mu_d,sigma_d,safety_stock,reorder_point
0,S002,P0002,12.601489,2.839430,12.395502,100.605927
1,S002,P0003,30.797012,7.990975,34.884518,250.463608
2,S002,P0004,9.407314,2.329668,10.170141,76.021339
3,S002,P0005,49.233761,7.041443,30.739347,375.375671
4,S002,P0006,19.522459,3.947262,17.231730,153.888947


In [69]:
holding_cost_per_unit = 0.5
stockout_cost_per_unit = 5

policy["holding_cost"] = policy["safety_stock"] * holding_cost_per_unit
policy["expected_stockout_cost"] = policy["sigma_d"] * 0.1 * stockout_cost_per_unit
policy["total_cost"] = policy["holding_cost"] + policy["expected_stockout_cost"]

policy.head()

,store_key,prod_key,mu_d,sigma_d,safety_stock,reorder_point,holding_cost,expected_stockout_cost,total_cost
0,S002,P0002,12.601489,2.839430,12.395502,100.605927,6.197751,1.419715,7.617466
1,S002,P0003,30.797012,7.990975,34.884518,250.463608,17.442259,3.995487,21.437746
2,S002,P0004,9.407314,2.329668,10.170141,76.021339,5.085071,1.164834,6.249905
3,S002,P0005,49.233761,7.041443,30.739347,375.375671,15.369674,3.520722,18.890396
4,S002,P0006,19.522459,3.947262,17.231730,153.888947,8.615865,1.973631,10.589496


In [70]:
sample_policy = policy.iloc[0]
sample_policy

,0
store_key,S002
prod_key,P0002
mu_d,12.601489
sigma_d,2.83943
safety_stock,12.395502
reorder_point,100.605927
holding_cost,6.197751
expected_stockout_cost,1.419715
total_cost,7.617466


In [71]:
import plotly.express as px

sample_store = sample_policy["store_key"]
sample_prod  = sample_policy["prod_key"]

sku_data = tmp[(tmp["store_key"] == sample_store) &
               (tmp["prod_key"] == sample_prod)]

fig = px.line(
    sku_data,
    x="date",
    y=["prediction"],
    title=f"Forecast for {sample_store} - {sample_prod}"
)

fig.show()

In [72]:
def simulate_service_level(service_level):
    z_map = {0.90: 1.28, 0.95: 1.65, 0.97: 1.88, 0.99: 2.33}
    Z = z_map.get(service_level, 1.65)

    temp = policy.copy()
    temp["safety_stock"] = Z * temp["sigma_d"] * np.sqrt(lead_time_days)
    temp["reorder_point"] = temp["mu_d"] * lead_time_days + temp["safety_stock"]

    return temp[["store_key","prod_key","safety_stock","reorder_point"]].head()

simulate_service_level(0.99)

,store_key,prod_key,safety_stock,reorder_point
0,S002,P0002,17.503952,105.714378
1,S002,P0003,49.261166,264.840240
2,S002,P0004,14.361472,80.212669
3,S002,P0005,43.407684,388.044006
4,S002,P0006,24.333288,160.990494


In [73]:
# Save dashboard artifacts
policy.to_csv("inventory_policy.csv", index=False)

# Keep a forecast table for interactive plots
forecast_cols = ["date", "store_key", "prod_key", "prediction"]
extra = [c for c in ["lower_95", "upper_95"] if c in tmp.columns]
tmp[forecast_cols + extra].to_csv("sku_forecasts.csv", index=False)

print("Saved: inventory_policy.csv, sku_forecasts.csv")

Saved: inventory_policy.csv, sku_forecasts.csv
